In [1]:
import sqlalchemy
import psycopg2
import os
import numpy
import ast
import pandas as pd
import json
import re
from lxml import etree
import numpy as np
import zipfile
from io import StringIO, BytesIO
import sumy
from sqlalchemy import create_engine
import seaborn
import nltk
nltk.download('punkt')

import matplotlib
import matplotlib.pylab as plt
%matplotlib inline

[nltk_data] Downloading package punkt to
[nltk_data]     /Users/melissaferrari/nltk_data...
[nltk_data]   Package punkt is already up-to-date!


In [2]:
congress = '114'
bill_type = 'hr'
default_bill_number = 4764# 1216 #
number = default_bill_number

In [ ]:
### Get XML doc from filepath

In [ ]:
bill_path_root = '/Users/melissaferrari/Projects/repo/congress/data/'

In [ ]:
bill_path = '{}{}/bills/{}/{}{}/text-versions/ih/'.format(bill_path_root, 
                                                          congress, 
                                                          bill_type, 
                                                          bill_type, 
                                                          number)
bill_path += 'BILLS-{}{}{}ih/xml'.format(congress, bill_type, number)

xml_file = os.listdir(bill_path)[0]

In [ ]:
tree = etree.parse(os.path.join(bill_path, xml_file))
string_tree = etree.tostring(tree).decode()

In [ ]:
### Query XML from database

In [ ]:
# bill_id = 'hr1216-114'
bill_id = '{}{}-{}'.format(bill_type, number, congress)
bill_id

In [ ]:
user = 'melissaferrari'  # add your Postgres username here
host = 'localhost'
dbname = 'congressional_bills'
db = create_engine('postgres://%s%s/%s' % (user, host, dbname))
con = None
con = psycopg2.connect(database=dbname, user=user)

In [ ]:
bill_id = 'hr2914-113'

In [ ]:
query_text = "SELECT bill_ix, text FROM bill_text bt INNER JOIN bills b ON bt.bill_ix = b.id WHERE b.bill_id = '%s';" % bill_id
query_text_results = pd.read_sql_query(query_text, con)

In [ ]:
query_text_results

In [ ]:
string_xml = query_text_results.loc[0]['text']

In [ ]:
string_xml = query_text_results.iloc[0]['text']
tree = etree.fromstring(string_xml)
#summarization_result = models.do_summarization(string_xml)

In [ ]:
tree = etree.fromstring(string_xml)

In [ ]:
query_summary = """
                SELECT * FROM summaries bt INNER JOIN bills b
                ON bt.bill_ix = b.id WHERE b.bill_id = '%s';
                """
data = (bill_id,)
query_summary_results = pd.read_sql_query(query_summary % data, con)
query_summary_results

In [ ]:
bill_id

In [ ]:
query_summary_results.loc[0, 'url']

In [ ]:
query_summary = """
                SELECT * FROM summaries bt INNER JOIN bills b
                ON bt.bill_ix = b.id WHERE b.bill_id = '%s';
                """
data = (bill_id,)
query_summary_results = pd.read_sql_query(query_summary % data, con)
query_summary_results

In [ ]:
print(query_summary_results.loc[0]['text'])

In [ ]:
query_info = "SELECT * FROM bills WHERE bill_id='%s';" % bill_id
query_info_results = pd.read_sql_query(query_info, con)
query_info_results.loc[0]
#_, official_title, short_title, subjects_top_term = query_info_results.iloc[0].values

In [ ]:
query_info = "SELECT bill_id, official_title, short_title, subjects_top_term FROM bills WHERE bill_id='%s';" % bill_id
query_info_results = pd.read_sql_query(query_info, con)
query_info_results
_, official_title, short_title, subjects_top_term = query_info_results.iloc[0].values

In [ ]:
tags = []
for elt in tree.getiterator():
    tag = elt.tag
    #if '}' in tag:
    #   tag = tag.split('}')[1]
    tags.append(tag)
#tags = list(np.unique(tags))
tags;

In [ ]:
title

In [ ]:
import spacy
nlp = spacy.load('en_core_web_lg')
doc = nlp(u'Apple is looking at buying U.K. startup for $1 billion')

for ent in doc.ents:
    print(ent.text, ent.start_char, ent.end_char, ent.label_)

In [ ]:
for elt in tree.getiterator('text'):
    if elt.text == None:
        pass
    if elt.tag == None:
        pass
    if elt.text == 'None':
        pass
    else:
        print(elt.text)

In [ ]:
def create_single_text_string(tree, tag='text'):
    text = ""
    for elt in tree.getiterator(tag):
        if isinstance(elt.text, str):
            text += elt.text  + ' '
    return text


In [ ]:
full_text = create_single_text_string(tree)
bad_string = "\n\t\t\t"
if bad_string in full_text:
    full_text =full_text.replace(bad_string, "")

In [ ]:
### Simple Gensim tutorial [[LINK](https://rare-technologies.com/text-summarization-with-gensim/)]

In [ ]:
import gensim #from gensim import summarization #import summarize, keywords

In [ ]:
len(full_text)

In [ ]:
summarized = gensim.summarization.summarize(full_text)

In [ ]:
print(summarized)

In [ ]:
len(summarized)/len(full_text)

In [ ]:
len(full_text) - len(summarized)

In [ ]:
kw = gensim.summarization.keywords(full_text)

In [ ]:
print(kw)

In [ ]:
# Implementing ROUGE
[GITHUB REPO](https://github.com/Diego999/py-rouge)

In [ ]:
import rouge

In [ ]:
bill_id

In [ ]:
query_info = "SELECT bill_id, official_title, short_title, subjects_top_term FROM bills WHERE bill_id='%s';" % bill_id
query_info_results = pd.read_sql_query(query_info, con)
query_info_results

In [ ]:
_, official_title, short_title, subjects_top_term = query_info_results.iloc[0].values

In [ ]:
query_summary = """
                SELECT bill_ix, text FROM summaries bt INNER JOIN bills b
                ON bt.bill_ix = b.id WHERE b.bill_id = '%s';
                """
data = (bill_id,)
query_summary_results = pd.read_sql_query(query_summary % data, con)

query_summary_results

In [ ]:
actual_summary = query_summary_results.iloc[0]['text']
# Remove summary title
to_remove = actual_summary.split('\n\n')[0]
if to_remove == short_title:
    print('REMOVED THE FOLLOWING TEXT: \n{}'.format(to_remove))
else:
    print('Did not remove first sentence: \n{}'.format(to_remove))
actual_summary = actual_summary.split(actual_summary.split('\n\n')[0] + '\n\n')[1]
actual_summary = actual_summary.replace('\n\n', ' ')

In [ ]:
actual_summary

In [ ]:
## DEFINE ROUGE EVALUATOR
evaluator = rouge.Rouge(metrics=['rouge-n', 'rouge-l', 'rouge-w'],
                        max_n=4,
                        limit_length=True,
                        length_limit=100,
                        length_limit_type='words',
                        alpha=0.5, # Default F1_score
                        weight_factor=1.2,
                        stemming=True)

In [ ]:
scores = evaluator.get_scores(summarized, actual_summary)

In [ ]:
try:
    rouge_df = pd.DataFrame(scores).transpose()
except:
    for metric, results in sorted(scores.items(), key=lambda x: x[0]):
        results = {key: value[0] for key, value in results[0].items()}
        scores[metric] = results
    rouge_df = pd.DataFrame(scores).transpose()
rouge_df

In [ ]:
rouge_df = np.multiply(rouge_df, 100.0)
rouge_df.columns = ['F1', 'P', 'R']
rouge_df

In [ ]:
def apply_rouge_naive(auto_summarization, actual_summarization, 
                      metrics=['rouge-n', 'rouge-l', 'rouge-w'],
                      max_n=4, weight_factor=1.2, stemming=True, 
                      percent=False):
    
    ## DEFINE ROUGE EVALUATOR
    evaluator = rouge.Rouge(metrics=metrics,
                            max_n=max_n,
                            limit_length=True,
                            length_limit=100,
                            length_limit_type='words',
                            alpha=0.5, # Default F1_score
                            weight_factor=weight_factor,
                            stemming=stemming)
    
    scores = evaluator.get_scores(auto_summarization, actual_summarization)
    try:
        rouge_df = pd.DataFrame(scores).transpose()
    except:
        for metric, results in sorted(scores.items(), key=lambda x: x[0]):
            results = {key: value[0] for key, value in results[0].items()}
            scores[metric] = results
        rouge_df = pd.DataFrame(scores).transpose()
    if percent:
        rouge_df = np.multiply(rouge_df, 100.0)
        rouge_df.columns = ['F1', 'P', 'R']
    return rouge_df

In [ ]:
### Random Summary Generator

In [ ]:
import random
import itertools
import time

In [ ]:
num_sentences_full = len(full_text.split('.'))
num_sentences_full

In [ ]:
target_summary_percentage = 0.25
num_sentences_sum = int(np.round(num_sentences_full*target_summary_percentage))
print('To achieve a summary that is approximately {}% we need provide a summary with {} sentences'.format(int(target_summary_percentage*100), num_sentences_sum))

In [ ]:
# choose random set of sentence locs 
sentence_locs = list(np.arange(1, num_sentences_full+1))
sentence_nums = random.sample(sentence_locs, num_sentences_sum)
sentence_nums = sorted(sentence_nums)

In [ ]:
# choose random set of sentence locs 
sentence_locs = list(np.arange(1, num_sentences_full+1))
sentence_nums = random.sample(sentence_locs, num_sentences_sum)
sentence_nums = sorted(sentence_nums)

In [ ]:
def calculate_factorial(n):
    total = n
    n = n-1 
    while n >=1:
        total = total*n
        n = n -1 
    return total

In [ ]:
def calculate_combinations(n, k):
    return int(np.divide(calculate_factorial(n), calculate_factorial(k)*calculate_factorial(n-k)))

In [ ]:
def calculate_permutations(n, k):
    return int(np.divide(calculate_factorial(n), calculate_factorial(n-k)))

In [ ]:
num_combinations = calculate_combinations(num_sentences_full, num_sentences_sum)
print('Number of {} combinations from set of {} elements is: {}'.format(num_sentences_sum, num_sentences_full, num_combinations))

In [ ]:
random_summaries = list(itertools.combinations(full_text.split('.'), num_sentences_sum))
random_summaries = ['. '.join(random_summary) for random_summary in random_summaries]

In [ ]:
len(random_summaries)

In [ ]:
rouge1_random_sum = pd.DataFrame()
for ix, random_summary in enumerate(random_summaries):
    rouge1_random_sum = rouge1_random_sum.append(apply_rouge_naive(random_summary, actual_summary, 
                                                                   metrics=['rouge-n'], max_n=1).set_index(pd.Index([ix])))

In [ ]:
random_summ_sentence_locs = list(itertools.combinations(np.arange(1, 18), 4))

In [ ]:
rouge1_random_stats = rouge1_random_sum[['f']].describe()
rouge1_random_stats

In [ ]:
fig, ax = plt.subplots()
rouge1_random_sum[['f']].hist(ax=ax);

In [ ]:
max_rouge = rouge1_random_stats.loc['max', 'f']
max_rouge

In [ ]:
min_rouge = rouge1_random_stats.loc['min', 'f']
min_rouge

In [ ]:
best_summary_ix = rouge1_random_sum.query('f == {}'.format(max_rouge)).index[0]
worst_summary_ix = rouge1_random_sum.query('f == {}'.format(min_rouge)).index[0]

In [ ]:
best_summary = random_summaries[best_summary_ix]
worst_summary = random_summaries[worst_summary_ix]

In [ ]:
best_summary_ix

In [ ]:
best_summary

In [ ]:
worst_summary

In [ ]:
random_summ_sentence_locs[worst_summary_ix]

In [ ]:
apply_rouge_naive(best_summary, actual_summary)

In [ ]:
rouge_random_summary = pd.DataFrame()
for ix, random_summary in enumerate(random_summaries):
    rouge_scores = apply_rouge_naive(random_summary, actual_summary, max_n=9)
    rouge_scores['random_sum_ix'] = ix
    rouge_random_summary = rouge_random_summary.append(rouge_scores)

In [ ]:
rouge_random_summary.head()

In [ ]:
rouge_f_summaries = rouge_random_summary[['random_sum_ix', 'f']]
rouge_f_summaries = rouge_f_summaries.reset_index()
rouge_f_summaries = rouge_f_summaries.rename(columns={'index':'metric'})
rouge_f_summaries = rouge_f_summaries.sort_values(['metric', 'random_sum_ix']).reset_index(drop=True)
rouge_f_summaries = rouge_f_summaries.set_index(['metric', 'random_sum_ix'])

In [ ]:
rouge_f_summaries.head()

In [ ]:
rouge_f_random_stats = rouge_f_summaries.groupby('metric').describe()
rouge_f_random_stats

In [ ]:
#rouge_f_random_stats.columns = rouge_f_random_stats.columns.get_level_values(1)

In [ ]:
#rouge_f_random_stats.iloc[:, rouge_f_random_stats.columns.get_level_values(1)=='max']

In [ ]:
metrics = list(rouge_f_summaries.index.get_level_values(0).unique())

In [ ]:
tester = pd.DataFrame()
tester = rouge_f_summaries.iloc[rouge_f_summaries.index.get_level_values(0) == 'rouge-1', :].copy()
tester = tester.reset_index() 
tester = tester.rename(columns={'f':tester.metric.unique()[0]})
del tester['metric']
del tester['random_sum_ix']

In [ ]:
for mindex_val in metrics[1:]:
    test = rouge_f_summaries.iloc[rouge_f_summaries.index.get_level_values(0) == mindex_val, :].copy()
    test = test.reset_index() 
    test = test.rename(columns={'f':test.metric.unique()[0]})
    del test['metric']
    del test['random_sum_ix']
    tester = tester.merge(test, left_index=True, right_index=True)

In [ ]:
fig, ax = plt.subplots(figsize=(16, 4))
seaborn.violinplot(data=tester[tester.columns], ax=ax);
ax.grid()

In [ ]:
stats = tester.describe()

In [ ]:
rouge_max = stats.loc[['max']].transpose()
rouge_max

In [ ]:
benchmark_rouge = rouge_f_random_stats.iloc[:, rouge_f_random_stats.columns.get_level_values(1) == 'max'].copy()
benchmark_rouge.columns = benchmark_rouge.columns.get_level_values(1)
del benchmark_rouge.index.name
benchmark_rouge

In [ ]:
for ix, max_val in rouge_max.iterrows():
    print('{} ix={} score={}'.format(ix,
                                      rouge_f_summaries.query("metric == '{}' & f == {}".format(ix, max_val[0])).reset_index()['random_sum_ix'][0], 
                                      np.round(max_val[0], 2)))

In [ ]:
idx_sort = np.argsort(tester.sum(axis=1))[::-1]

In [ ]:
fig, ax = plt.subplots()
ax.plot(list(tester.loc[idx_sort].sum(axis=1)));
ax.set_ylim(bottom=0); ax.grid();

In [ ]:
fig, ax = plt.subplots()
tester.loc[idx_sort, metrics].reset_index(drop=True).plot(colormap='viridis', ax=ax);
ax.legend(loc='center left', bbox_to_anchor=(1.0, 0.5)); ax.grid();

In [ ]:
rouge_1_ix_sort = list(np.argsort(tester[['rouge-1']], axis=0)[::-1]['rouge-1'].values)
rouge_l_ix_sort = list(np.argsort(tester[['rouge-l']], axis=0)[::-1]['rouge-l'].values)
rouge_w_ix_sort = list(np.argsort(tester[['rouge-w']], axis=0)[::-1]['rouge-w'].values)

In [ ]:
rouge_1_ix_sort[:10]

In [ ]:
rouge_l_ix_sort[:10]

In [ ]:
rouge_w_ix_sort[:10]

In [ ]:
random_summaries[1103]

In [ ]:
random_summaries[1674]

In [ ]:
# Adding Complexity
Now that I have a very minimal model that runs, I need to add complexity to the model. 

In [ ]:
import sumy

In [ ]:
from sumy.parsers.html import HtmlParser
from sumy.parsers.plaintext import PlaintextParser
from sumy.nlp.tokenizers import Tokenizer
from sumy.summarizers.lsa import LsaSummarizer
from sumy.nlp.stemmers import Stemmer
from sumy.utils import get_stop_words

In [ ]:
num_sentences_full = len(full_text.split('.'))
print('Full text is {} sentences'.format(num_sentences_full))

In [ ]:
target_summary_percentage = 0.25
num_sentences_sum = int(np.round(num_sentences_full*target_summary_percentage))
print('To achieve a summary that is approximately {}% we need provide a summary with {} sentences'.format(int(target_summary_percentage*100), num_sentences_sum))

In [ ]:
### Latent Semantic Analysis.
Combines term freq with SVD (singular value decomposition)

In [ ]:
#### Without stemming

In [ ]:
summarizer = LsaSummarizer()

In [ ]:
sumy_lsa_summarized = ""
full_text_parsed = PlaintextParser.from_string(full_text, Tokenizer('english'))
for sentence in summarizer(full_text_parsed.document, num_sentences_sum):
    #print(sentence)
    #print(' ')
    sumy_lsa_summarized += str(sentence) + " "

In [ ]:
sumy_lsa_summarized

In [ ]:
sumy_lsa_summarized_rouge = apply_rouge_naive(sumy_lsa_summarized, actual_summary, 
                                              max_n=4, weight_factor=1.2, stemming=True)
sumy_lsa_summarized_rouge

In [ ]:
#### With stemming.

In [ ]:
language = 'English'
stemmer_eng = Stemmer(language)

In [ ]:
summarizer = LsaSummarizer(stemmer_eng)
summarizer.stop_words = get_stop_words(language)

In [ ]:
sumy_lsa_summarized_stemmed = ""
full_text_parsed = PlaintextParser.from_string(full_text, Tokenizer('english'))
for sentence in summarizer(full_text_parsed.document, num_sentences_sum):
    #print(sentence)
    #print(' ')
    sumy_lsa_summarized_stemmed += str(sentence) + " "

In [ ]:
sumy_lsa_summarized_stemmed_rouge = apply_rouge_naive(sumy_lsa_summarized_stemmed, actual_summary, 
                                              max_n=4, weight_factor=1.2, stemming=True)
sumy_lsa_summarized_stemmed_rouge

In [ ]:
actual_summary

In [ ]:
print("\n\n".join(sumy_lsa_summarized_stemmed.split('. ')))

In [ ]:
### Lex Rank
Graphical based text summarizer
[SOURCE](https://medium.com/@ondenyi.eric/extractive-text-summarization-techniques-with-sumy-3d3b127a0a32)

In [ ]:
from sumy.summarizers.lex_rank import LexRankSummarizer 

In [ ]:
summarizer = LexRankSummarizer()

In [ ]:
sumy_summarized_lexrank = ""
full_text_parsed = PlaintextParser.from_string(full_text, Tokenizer('english'))
for sentence in summarizer(full_text_parsed.document, num_sentences_sum):
    sumy_summarized_lexrank += str(sentence) + " "

In [ ]:
print(sumy_summarized_lexrank)

In [ ]:
apply_rouge_naive(sumy_summarized_lexrank, actual_summary)

In [ ]:
### Luhn
Scored based on freq of most important words. [SOURCE](https://medium.com/@ondenyi.eric/extractive-text-summarization-techniques-with-sumy-3d3b127a0a32)

In [ ]:
from sumy.summarizers.luhn import LuhnSummarizer
summarizer = LuhnSummarizer()
sumy_summarized_luhn = ""
for sentence in summarizer(full_text_parsed.document, num_sentences_sum):
    sumy_summarized_luhn += str(sentence) + " "

In [ ]:
print("\n\n".join(sumy_summarized_luhn.split('. ')))

In [ ]:
apply_rouge_naive(sumy_summarized_luhn, actual_summary)

In [ ]:
### Text Rank
Graph-based summarization technique with keyword extraction.

#### MUST also implement via pytextrank

In [ ]:
from sumy.summarizers.text_rank import TextRankSummarizer
summarizer = TextRankSummarizer()
sumy_summarized_textrank = ""
for sentence in summarizer(full_text_parsed.document, num_sentences_sum):
    sumy_summarized_textrank += str(sentence) + " "

In [ ]:
sumy_summarized_textrank

In [ ]:
apply_rouge_naive(sumy_summarized_textrank, actual_summary)

In [ ]:
### TextRank with Pytextrank

In [ ]:
import pytextrank

In [ ]:
#for ix, sentence in enumerate(full_text.split('.')):

In [ ]:
text_dict = {"id":"1", "text":full_text}
text_json = json.dumps(text_dict)

In [ ]:
for graf in pytextrank.parse_doc(text_dict):
    print(graf._asdict())

In [ ]:
### TextRank with Summa

In [ ]:
import summa

In [ ]:
### TextRank with NLTK.
[Source](https://www.analyticsvidhya.com/blog/2018/11/introduction-text-summarization-textrank-python/)

In [ ]:
import numpy as np
import pandas as pd
import nltk
nltk.download('punkt') # one time execution
import re

In [ ]:
#### 1. Split into sentences.

In [ ]:
sentences = nltk.tokenize.sent_tokenize(full_text)

In [ ]:
#### 2. GloVe Word Embeddings (vector representation of words)

In [ ]:
!wget http://nlp.stanford.edu/data/glove.6B.zip
!unzip glove*.zip

In [ ]:
# Extract word vectors
f = open('../nlp_models/glove.6B/glove.6B.300d.txt', encoding='utf-8')
word_embeddings = {}
for line in f:
    values = line.split()
    word = values[0]
    coefs = np.asarray(values[1:], dtype='float32')
    word_embeddings[word] = coefs
f.close()

In [ ]:
len(word_embeddings)

In [ ]:
#### 3. Text preprocessing.

In [ ]:
# remove punctuations, numbers and special characters
clean_sentences = pd.Series(sentences).str.replace("[^a-zA-Z]", " ")

# make alphabets lowercase
clean_sentences = [s.lower() for s in clean_sentences]

In [ ]:
clean_sentences

In [ ]:
clean_sentences[:2]

In [ ]:
nltk.download('stopwords')

In [ ]:
from nltk.corpus import stopwords
stop_words = stopwords.words('english')

In [ ]:
# function to remove stopwords
def remove_stopwords(sen):
    sen_new = " ".join([i for i in sen if i not in stop_words])
    return sen_new

In [ ]:
# remove stopwords from the sentences
clean_sentences = [remove_stopwords(r.split()) for r in clean_sentences]

In [ ]:
#### 4. Vector representation of sentences.

In [ ]:
sentence_vectors = []
for i in clean_sentences:
    if len(i) != 0:
        v = sum([word_embeddings.get(w, np.zeros((100,))) for w in i.split()])/(len(i.split())+0.001)
    else:
        v = np.zeros((100,))
    sentence_vectors.append(v)

In [ ]:
#### 5. Similarity Matrix.
Use cosine similarity to find similarities between sentences.

In [ ]:
# similarity matrix
sim_mat = np.zeros([len(sentences), len(sentences)])

In [ ]:
np.shape(sim_mat)

In [ ]:
from sklearn.metrics.pairwise import cosine_similarity

In [ ]:
i = 4
j = 15

In [ ]:
clean_sentences[i]

In [ ]:
clean_sentences[j]

In [ ]:
np.shape(sentence_vectors[i])

In [ ]:
np.shape(sentence_vectors[i].reshape(1,100))

In [ ]:
for i in range(len(sentences)):
    for j in range(len(sentences)):
        if i != j:
            sim_mat[i][j] = cosine_similarity(sentence_vectors[i].reshape(1,100), 
                                              sentence_vectors[j].reshape(1,100))[0,0]

In [ ]:
#### 6. Apply pagerank.

In [ ]:
import networkx as nx

In [ ]:
# Similarity matrix to graph
nx_graph = nx.from_numpy_array(sim_mat)

In [ ]:
scores = nx.pagerank(nx_graph)

In [ ]:
#### 7. Summary extraction.

In [ ]:
ranked_sentences = sorted(((scores[i],s) for i,s in enumerate(sentences)), reverse=True)

In [ ]:
nltk_summarized_textrank = ""
for i in range(num_sentences_sum):
    nltk_summarized_textrank += str(ranked_sentences[i][1]) + " "

In [ ]:
nltk_summarized_textrank

In [ ]:
rouge_nltk_textrank = apply_rouge_naive(nltk_summarized_textrank, actual_summary, max_n=9)[['f']]
rouge_nltk_textrank

In [ ]:
np.divide(rouge_nltk_textrank['f'].values, benchmark_rouge['max'].values)

In [ ]:
### Supervised learning approach
[Supervised Machine Learning for Summarizaing Legal Documents](http://atour.iro.umontreal.ca/rali/sites/default/files/publis/SupervisedCanAI2010.pdf)

In [ ]:
ordered_tags = []
for elt in tree.getiterator():
    tag = elt.tag
    if '}' in tag:
        tag = tag.split('}')[1]
    ordered_tags.append(tag)
unique_tags = list(np.unique(tags))

In [ ]:
for elt in tree.getiterator():
    print('{} --> {}'.format(elt.tag, elt.text))

In [ ]:
ordered_tags[:10]

In [ ]:
#### Unsupervised Text Summarization
[SOURCE](https://medium.com/jatana/unsupervised-text-summarization-using-sentence-embeddings-adb15ce83db1)

In [ ]:
from sklearn import cluster, metrics

In [ ]:
kmeans = cluster.KMeans(n_clusters=num_sentences_sum)

In [ ]:
kmeans_output = kmeans.fit(sentence_vectors)

In [ ]:
from sklearn import metrics

In [ ]:
avg = []
for j in range(num_sentences_sum):
    idx = np.where(kmeans.labels_ == j)[0]
    avg.append(np.mean(idx))

In [ ]:
cluster_centers = kmeans.cluster_centers_

In [ ]:
closest, distances = metrics.pairwise_distances_argmin_min(cluster_centers, 
                                                           sentence_vectors)

In [ ]:
ordering = sorted(range(num_sentences_sum), key=lambda k: avg[k])

In [ ]:
summary = ' '.join([sentences[closest[idx]] for idx in ordering])

In [ ]:
summary

In [ ]:
apply_rouge_naive(summary, actual_summary)

In [ ]:
### Ideas

In [ ]:
query_text = "SELECT bill_ix, text FROM bill_text;"
query_text_results = pd.read_sql_query(query_text, con)

In [ ]:
def create_single_text_string(tree, tag='text'):
    text = ""
    for elt in tree.getiterator(tag):
        if isinstance(elt.text, str):
            text += elt.text  + ' '
    return text

In [ ]:

def do_summarization(string_xml):
    string_length = len(string_xml)
    print('text length: {}'.format(string_length))
    tree = etree.fromstring(string_xml)

    text = create_single_text_string(tree, tag='text')

    summarized = summarization.summarize(text)
    print('summary length: {}'.format(len(summarized)))

    return summarized

In [ ]:
string_xml = query_text_results.iloc[0]['text']
tree = etree.fromstring(string_tree)
#summarization_result = models.do_summarization(string_xml)
full_text = create_single_text_string(tree)

In [ ]:
tree = etree.parse(os.path.join(bill_path, xml_file))
string_tree = etree.tostring(tree).decode()

In [ ]:
query_text_results['tree'] = query_text_results['text'].apply(etree.fromstring)


In [ ]:
query_text_results['full_text'] = query_text_results['tree'].apply(create_single_text_string)

In [ ]:
query_text_results.head()

In [ ]:
query_text_results['tokenize'] = query_text_results['full_text'].apply(sent_tokenize)

In [ ]:
query_text_results['num_sentences'] = query_text_results['tokenize'].apply(len).astype(int)

In [ ]:
query_text_results['num_sentences'].describe()

In [ ]:
fig, ax = plt.subplots(figsize=(16, 6))
query_text_results['num_sentences'].hist(bins=100, ax=ax, alpha=0.7);
ax.set_yscale('log'); #ax.set_xlim(xmin=0);
ax.set_xlabel('Number of Sentences', size=14);
ax.set_title('Log Distribution of Bill Length (N=5550)', size=20);

In [ ]:
query_text_results[query_text_results['num_sentences'] > query_text_results['num_sentences'].describe()['50%']]

In [ ]:
long_bill = query_text_results.loc[query_text_results['num_sentences'] == 
                                         query_text_results['num_sentences'].describe()['max']]

In [ ]:
longest_bill_ix = query_text_results.loc[query_text_results['num_sentences'] == 
                                         query_text_results['num_sentences'].describe()['max'], 'bill_ix'].values[0]

In [ ]:
longest_bill = query_text_results.loc[query_text_results['num_sentences'] == 
                                      query_text_results['num_sentences'].describe()['max'], 'tokenize'].values[0]

In [ ]:
query_text_results.loc[query_text_results['num_sentences'] == 
                                      query_text_results['num_sentences'].describe()['max']]

In [ ]:
official_title

In [ ]:
longest_bill_ix

In [ ]:
query_info = "SELECT bill_id, official_title, short_title, subjects_top_term FROM bills WHERE id='%s';" % longest_bill_ix
query_info_results = pd.read_sql_query(query_info, con)

In [ ]:
bill_id, official_title, short_title, subjects_top_term = query_info_results.loc[0].values

In [ ]:
bill_id

In [ ]:
query_summary = """
                SELECT bill_ix, text FROM summaries bt INNER JOIN bills b
                ON bt.bill_ix = b.id WHERE b.bill_id = '%s';
                """

In [ ]:
data = (bill_id,)
query_summary_results = pd.read_sql_query(query_summary % data, con)
query_summary_results

In [ ]:
 query_summary_results.loc[0].to_dict()

In [ ]:
summary_dict = query_summary_results.loc[0].to_doct)

In [ ]:
tree = etree.fromstring(long_bill.loc[25, 'text'])

In [ ]:
ordered_tags = []
for elt in tree.getiterator():
    tag = elt.tag
    if '}' in tag:
        tag = tag.split('}')[1]
    ordered_tags.append(tag)
unique_tags = list(np.unique(tags))

In [ ]:
unique_tags[:10]

In [ ]:
ordered_tags[:10]

In [ ]:
full_sent = nltk.tokenize.sent_tokenize(full_text)
summ_sent = nltk.tokenize.sent_tokenize(actual_summary)

In [ ]:
full_sent

In [ ]:
sim_mat = np.zeros([len(full_sent), len(summ_sent)])

In [ ]:
from sklearn.metrics.pairwise import cosine_similarity

In [ ]:
full_sent_clean = pd.Series(full_sent).str.replace("[^a-zA-Z]", " ")

In [ ]:
summ_sent_clean = pd.Series(summ_sent).str.replace("[^a-zA-Z]", " ")

In [ ]:
# make alphabets lowercase
summ_sent_clean = [s.lower() for s in summ_sent_clean]
full_sent_clean = [s.lower() for s in full_sent_clean]

In [ ]:
from nltk.corpus import stopwords

In [ ]:
stop_words = nltk.corpus.stopwords.words('english')
# function to remove stopwords
def remove_stopwords(sen):
    sen_new = " ".join([i for i in sen if i not in stop_words])
    return sen_new

In [ ]:
# remove stopwords from the sentences
summ_sent_clean = [remove_stopwords(r.split()) for r in summ_sent_clean]
full_sent_clean = [remove_stopwords(r.split()) for r in full_sent_clean]

In [ ]:
v = sum([word_embeddings.get(w, np.zeros((100,))) for w in i.split()])/(len(i.split())+0.001)

In [ ]:
summ_sentence_vectors = []
for i in summ_sent_clean:
    if len(i) != 0:
        v = sum([word_embeddings.get(w, np.zeros((100,))) for w in i.split()])/(len(i.split())+0.001)
    else:
        v = np.zeros((100,))
    summ_sentence_vectors.append(v)
full_sentence_vectors = []
for i in full_sent_clean:
    if len(i) != 0:
        v = sum([word_embeddings.get(w, np.zeros((100,))) for w in i.split()])/(len(i.split())+0.001)
    else:
        v = np.zeros((100,))
    full_sentence_vectors.append(v)

In [ ]:
np.shape(sim_mat)

In [ ]:
sim_mat[5][1]

In [ ]:
np.shape(full_sentence_vectors)

In [ ]:
for i in range(len(full_sent)):
    for j in range(len(summ_sent)):
            sim_mat[i][j] = cosine_similarity(full_sentence_vectors[i].reshape(1,100), 
                                              summ_sentence_vectors[j].reshape(1,100))[0,0]

In [ ]:
sim_mat

In [ ]:
type(sim_mat)

In [ ]:
ix_match = np.argmax(sim_mat, axis=0)
ix_match

In [ ]:
sim_mat[:, 0]

In [ ]:
sim_mat[8, 0]

In [ ]:
sim_mat[:, 1]

In [ ]:
sim_mat[5, 1]

In [ ]:
closest_match = [full_sent[i] for i in ix_match]
closest_match

In [ ]:
from nltk.corpus import stopwords 
from nltk.stem.wordnet import WordNetLemmatizer
import string
stop = set(stopwords.words('english'))
exclude = set(string.punctuation) 
lemma = WordNetLemmatizer()
def clean(doc):
    stop_free = " ".join([i for i in doc.lower().split() if i not in stop])
    punc_free = ''.join(ch for ch in stop_free if ch not in exclude)
    normalized = " ".join(lemma.lemmatize(word) for word in punc_free.split())
    return normalized

In [ ]:
doc_clean = [clean(doc).split() for doc in doc_complete]   

In [ ]:
# Named entities.

In [ ]:
import spacy
nlp = spacy.load('en_core_web_lg')

In [ ]:
import spacy
nlp = spacy.load('en_core_web_lg')
get_ipython().config.get('IPKernelApp', {})['parent_appname'] = ""

In [ ]:
def create_single_text_string(tree, tag='text'):
    text = ""
    for elt in tree.getiterator(tag):
        if isinstance(elt.text, str):
            text += elt.text  + ' '
    return text

In [ ]:

full_text = create_single_text_string(tree)
bad_string = "\n\t\t\t"
if bad_string in full_text:
    full_text =full_text.replace(bad_string, "")

In [ ]:
for elt in tree.getiterator('text'):
    print(elt.tag, elt.text)

In [ ]:
full_text = full_text.replace("\t", " ")

In [ ]:
doc = nlp(full_text)
sentences = [sent.string.strip() for sent in doc.sents]

In [ ]:
import nltk

In [ ]:
full_sent = nltk.tokenize.sent_tokenize(full_text)

In [ ]:
doc.ents

In [ ]:
df_ent = pd.DataFrame()
for ix, sentence in enumerate(full_sent):
    doc = nlp(sentence)

    for ent in doc.ents:
        # print(ix, ent.text, ent.start_char, ent.end_char, ent.label_)
        df_ent = df_ent.append([[ix, ent.text, ent.start_char, ent.end_char, ent.label_]], ignore_index=True)
df_ent.columns = ['sentence', 'text', 'start_char', 'end_char', 'label']

In [ ]:
df_ent

In [ ]:
entities